### 6. 모델링
- 모델 고도화
- 모델 예측값 시각화

### 1. 데이터 읽어오기

In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')      
submission = pd.read_csv('sample_submission.csv')

### 2. 데이터 확인

In [ ]:
train.head()

In [ ]:
# 데이터 결측치 및 변수들의 타입 확인
train.isnull().sum()

### 3. 데이터 전처리

In [ ]:
features = ['vh_id', 'now_longitude', 'now_arrive_time', 'distance', 'next_longitude']

# 학습에 사용할 변수 X 할당
X = train[features]   

# 예측할 변수 y 할당
y = train['next_arrive_time']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.3, shuffle=False)

### 4. 데이터 학습

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=20, criterion='squared_error', random_state=42)

In [ ]:
# 모델 학습
rf_model.fit(X_train, y_train)

# 예측값 획득
predict = rf_model.predict(X_valid)
predict

In [ ]:
from sklearn.metrics import mean_squared_error

def rmse(real, pred):    
    return mean_squared_error(real, pred)**0.5

RMSE = rmse(y_valid, predict)
RMSE

In [ ]:
# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트를 사용하기 위한 코드
fe = fm.FontEntry(fname = 'NotoSansKR-Regular.otf', name = 'NotoSansKR')
fm.fontManager.ttflist.insert(0, fe)
plt.rc('font', family='NotoSansKR')

RMSE = rmse(y_valid, predict)

plt.figure(dpi=150)

plt.title('RMSE : ' + str(RMSE)[:8])

plt.plot(y_valid.reset_index(drop=True), alpha=0.6, label='real')
plt.plot(predict, alpha=0.6, label='pred')

plt.legend()
plt.show()

In [ ]:
import xgboost

model = xgboost.XGBRegressor(objective='reg:squarederror', n_estimators = 3000, eval_metric = 'rmse', early_stopping_rounds=10, random_state=42)

In [ ]:
model.fit(X_train,y_train, eval_set=[(X_valid,y_valid)], verbose=10)

y_pred = model.predict(X_valid)

In [ ]:
RMSE = rmse(y_valid, y_pred)
RMSE

In [ ]:
RMSE = rmse(y_valid, y_pred)

# figure 및 ax 생성
fig, ax = plt.subplots(dpi=150)

# 그래프 제목 설정
ax.set_title('RMSE : ' + str(RMSE)[:8])

# 실제 값과 예측 값 그래프 그리기
ax.plot(y_valid.reset_index(drop=True), alpha=0.6, label='real')
ax.plot(y_pred, alpha=0.6, label='pred')

# 범례 추가
ax.legend()

# 그래프 출력
plt.show()

In [ ]:
# 1. 모델 정의
final_model = xgboost.XGBRegressor(objective='reg:squarederror', n_estimators = 3000, eval_metric = 'rmse', early_stopping_rounds=10)

# 2. 모델 학습
final_model.fit(X_train, y_train, eval_set=[(X_valid,y_valid)], verbose= 10)

# 3. 예측
xgb_predict = final_model.predict(test[features])

# 예측값 시각화
plt.plot(xgb_predict)
plt.show()

### 5. CSV파일로 저장

In [ ]:
submission['next_arrive_time'] = predict
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False, quoting=2, encoding="utf-8-sig", na_rep=0, float_format='%.6f')